# 第14章：もうちょっとだけモナド

Maybeモナド・リストもなど・IOモナドを今までやってきた  
さらにいくつかのモナドがこの章で出てくる  

この章で出てくるのはmtlパッケージ  
インストールされているかは `ghc-pkg list` とうってmtlとバージョン番号が出るはず  

### Writerの話

In [1]:
isBigGang :: Int -> Bool
isBigGang x = x > 9

In [2]:
isBigGang :: Int -> (Bool, String)
isBigGang x = (x > 9, "Compared gang size to 9.")

In [3]:
isBigGang 3

(False,"Compared gang size to 9.")

例えば既に文脈がついた値 `(3, "Smallish gang.")` をisBigGangに食わせたかったらどうするか  
ログのついた値、`(a, String)` 型の値と `a -> (b, String)` 型の関数2つを取り、その値を関数の方に食わせる関数を作る  

In [4]:
applyLog :: (a, String) -> (a -> (b, String)) -> (b, String)
applyLog (x, log) f = let (y, newLog) = f x in (y, log ++ newLog)

In [6]:
(3, "Smallish gang.") `applyLog` isBigGang

(False,"Smallish gang.Compared gang size to 9.")

In [7]:
("Tobin", "Got outlaw name.") `applyLog` (\x -> (length x, "applied length"))

(5,"Got outlaw name.applied length")

### モノイドが助けに来たよ

Logは別にStringである必要はない  


In [8]:
applyLog :: (Monoid m) => (a, m) -> (a -> (b, m)) -> (b, m)
applyLog (x, log) f = let (y, newLog) = f x in (y, log `mappend` newLog)


In [9]:
import Data.Monoid

type Food = String
type Price = Sum Int

addDrink :: Food -> (Food, Price)
addDrink "beans" = ("milk", Sum 25)
addDrink "jerky" = ("whiskey", Sum 99)
addDrink _ = ("beer", Sum 30)


In [10]:
Sum 3 `mappend` Sum 9

Sum {getSum = 12}